In [ ]:
# Black–Litterman Bayesian Portfolio Optimizer
This notebook demonstrates fetching data, computing historical MV statistics, building Black–Litterman posterior, and plotting efficient frontiers.

# Black–Litterman Bayesian Portfolio Optimizer
This notebook demonstrates fetching data, computing historical MV statistics, building Black–Litterman posterior, and plotting efficient frontiers.

In [ ]:
# Imports
import pandas as pd
from src import data_loader as dl
from src import implied_returns as ir
from src import views as vw
from src import optimizer as opt
from src import plots
import matplotlib.pyplot as plt

In [ ]:
# Fetch prices
tickers = ['AAPL','MSFT','GOOGL','AMZN']
prices = dl.fetch_prices(tickers, start='2018-01-01').prices
prices.tail()

In [ ]:
# Historical mu and Sigma
mu_hist, Sigma_hist = dl.returns_and_covariance(prices, method='log')
mu_hist, Sigma_hist.shape

In [ ]:
# Market caps -> weights and equilibrium returns
market_caps = {t: 1.0 for t in tickers}  # placeholder
w_mkt = ir.market_caps_to_weights(market_caps)
delta = 2.5
pi = ir.compute_pi(Sigma_hist, w_mkt, delta)
pi

In [ ]:
code
python
# Optimal (max Sharpe) portfolios under MV and BL
w_mv, r_mv, s_mv, sh_mv = opt.max_sharpe_portfolio(mu_hist, Sigma_hist)
w_bl, r_bl, s_bl, sh_bl = opt.max_sharpe_portfolio(bl.mu_bl, bl.Sigma_bl)
print('MV max Sharpe:', {'ret': r_mv, 'risk': s_mv, 'sharpe': sh_mv})
print(w_mv.sort_values(ascending=False))
print('BL max Sharpe:', {'ret': r_bl, 'risk': s_bl, 'sharpe': sh_bl})
print(w_bl.sort_values(ascending=False))
# Views: AAPL expected to beat MSFT by 2% annually
P_row, q = vw.relative_view(tickers, long='AAPL', short='MSFT', magnitude=0.02)
P, Q = vw.build_PQ([(P_row, q)])
bl = opt.black_litterman_posterior(Sigma_hist, pi, P, Q, tau=0.05)
bl.mu_bl

In [ ]:
# Frontiers and plot
curves = opt.compare_mv_vs_bl(mu_hist, Sigma_hist, bl.mu_bl, bl.Sigma_bl)
fig, ax = plots.plot_frontiers(*curves.values())
plt.show()